In [ ]:
import pandas as pd
from sklearn import preprocessing

In [ ]:
holidays_events = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv', index_col='date')
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', index_col='id')
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', index_col='id')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [ ]:
holidays_events.head()

In [ ]:
#holidays_count_date = pd.DataFrame(holidays_events.date.value_counts(), dtype='int8')

In [ ]:
#holidays_count_date = holidays_count_date.reset_index()
#holidays_count_date.rename(columns = {'index': 'date', 'date': 'count'}, inplace = True)
#holidays_count_date

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
test.head()

In [ ]:
train1 = train.copy()
test1 = test.copy()

In [ ]:
train1.head()

In [ ]:
train1 = pd.merge(train1, oil, how="left", on="date")
test1 = pd.merge(test1, oil, how="left", on="date")
len(train1)

In [ ]:
#train1 = pd.merge(train1, holidays_count_date, how="left", on="date")
#test1 = pd.merge(test1, holidays_count_date, how="left", on="date")
#len(train1)

In [ ]:
train1 = pd.merge(train1, transactions, how="left", on=["date",'store_nbr'])
test1 = pd.merge(test1, transactions, how="left", on=["date",'store_nbr'])
len(train1)

In [ ]:
train1.head()

In [ ]:
train1 = pd.merge(train1, stores, how="left", on=["store_nbr"])
test1 = pd.merge(test1, stores, how="left", on=["store_nbr"])
train1 = train1.fillna(0)
test1 = test1.fillna(0)

In [ ]:
len(train1)

In [ ]:
train1.head()

In [ ]:
train1['onpromotion_date_store'] = train1.groupby(['date','store_nbr'])['onpromotion'].transform("sum")   
test1['onpromotion_date_store'] = test1.groupby(['date','store_nbr'])['onpromotion'].transform("sum")   

In [ ]:
#train1['diff1_onpromotion'] = train1.onpromotion.diff(periods=1684)
#test1['diff1_onpromotion'] = test1.onpromotion.diff(periods=1684)

In [ ]:
#train1['diff2_onpromotion'] = train1.onpromotion.diff(periods=1684*2)
#test1['diff2_onpromotion'] = test1.onpromotion.diff(periods=1684*2)

In [ ]:
train2 = train1.copy()
test2 = test1.copy()

In [ ]:
train2.head()

In [ ]:
len(train2)

# Extract date features

In [ ]:
def extract_weekday(s):
    return s.dayofweek

def extract_monthday(s):
    return s.day

def extract_month(s):
    return s.month

def extract_year(s):
    return s.year

In [ ]:
train2['date'] = pd.to_datetime(train2['date'])
train2['weekday'] = train2['date'].apply(extract_weekday)
train2['extract_monthday'] = train2['date'].apply(extract_monthday)
train2['year'] = train2['date'].apply(extract_year)
train2['month'] = train2['date'].apply(extract_month)

In [ ]:
train2.year.describe()

In [ ]:
train2.head()

In [ ]:
test2['date'] = pd.to_datetime(test2['date'])
test2['weekday'] = test2['date'].apply(extract_weekday) 
test2['extract_monthday'] = test2['date'].apply(extract_monthday)
test2['year'] = test2['date'].apply(extract_year)
test2['month'] = test2['date'].apply(extract_month)

In [ ]:
test2.year.describe()

In [ ]:
train2.dtypes

# Reduce memory usage

In [ ]:
for col in train2.columns:
    if train2[col].dtype == 'float64' and col != 'sales':
        train2[col] = train2[col].astype('float32')
        test2[col] = test2[col].astype('float32')
    if train2[col].dtype == 'int64':
        train2[col] = train2[col].astype('int8')
        test2[col] = test2[col].astype('int8')


In [ ]:
train2.nunique()

In [ ]:
train2.fillna(0)
test2.fillna(0)

In [ ]:
cat_cols = [cname for cname in train2.columns if train2[cname].dtype == 'object']
cat_cols

In [ ]:
y = pd.DataFrame(train2.sales)
train2 = train2.drop(['sales'], axis=1)

In [ ]:
enc = preprocessing.LabelEncoder()
for col in cat_cols:
    train2[col] = enc.fit_transform(train2[col].astype(str))
    test2[col] = enc.fit_transform(test2[col])

In [ ]:
test2.head()

In [ ]:
len(test2.iloc[0])

In [ ]:
import tensorflow as tf

In [ ]:
# Detect TPU, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
train2 = train2.drop('date', axis=1)
test2 = test2.drop('date', axis=1)

In [ ]:
#date = list(train2.groupby('date'))
#1684 timeseries

In [ ]:
import numpy as np
train_data = np.array(train2)
test_data = np.array(test2)

In [ ]:
train_data.ndim

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
qt = QuantileTransformer(n_quantiles=300, output_distribution='uniform')
train_data = qt.fit_transform(train_data)
test_data = qt.transform(test_data)

In [ ]:
qty = QuantileTransformer(n_quantiles=300, output_distribution='uniform')
y = qty.fit_transform(y)

# deep network model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x_scaler = MinMaxScaler()
dataset_train_sc = x_scaler.fit_transform(train_data)
dataset_test_sc = x_scaler.transform(test_data)

In [ ]:
y_scaler = MinMaxScaler()
y_sc = y_scaler.fit_transform(y)

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
#from tensorflow.keras.engine.input_layer import Input
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import BatchNormalization

In [ ]:
lookback = 1
batch_size = 8192

In [ ]:
def model_builder(lr):
    """Модель нейронной сети"""
    inputA = keras.Input(shape=(15))
    line = Reshape((15,1))(inputA)
    line = Conv1D(filters=256, kernel_size=1, padding='same', activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.1)(line)

    line = MaxPooling1D(pool_size=2)(line)
    
    line = Conv1D(filters=512, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.3)(line)
    
    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Conv1D(filters=1024, kernel_size=2, activation='relu')(line)
    line = BatchNormalization()(line)
    line = Dropout(0.3)(line)

    #line = MaxPooling1D(pool_size=2)(line)
    
    line = Flatten()(line)
    line = Dense(512, activation='relu')(line)
    line = Dense(256, activation='relu')(line)
    line = Dense(128, activation='relu')(line)
    
    #line = Dropout(0.3)(line)
    #line = Dropout(0.3)(line)
    
    outputA = Dense(units=1)(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(
        #loss = tf.keras.losses.MeanSquaredLogarithmicError(reduction="auto", name="mean_squared_logarithmic_error"),
        loss = tf.keras.losses.MeanSquaredError(name='mse'),
        optimizer = Adam(lr=lr), metrics=['mae'],)
    return model

def model_builder1(lr):
    """Модель нейронной сети"""
    inputA = keras.Input(shape=(lookback, 11))
    line = tf.keras.layers.LSTM(16)(inputA)
    
    line = Dense(128, activation='relu')(line)
    line = Dropout(0.5)(line)
    #line = Dense(64, activation='relu')(line)
    outputA = Dense(units=1)(line)
    model = Model(inputs=inputA, outputs=outputA)
    #model = keras.models.load_model('models/model2')
    model.compile(
        loss = tf.keras.losses.MeanSquaredLogarithmicError(reduction="auto", name="mean_squared_logarithmic_error"),
        optimizer = Adam(lr=lr), metrics=['mae'],)
    return model

In [ ]:
lr=0.001
with strategy.scope():
    model = model_builder(lr)
#with strategy.scope():
#    model = load_model('../input/ss-tsf/best_6.5913e-04-170.h5')

In [ ]:
model.summary()

In [ ]:
checkpoint_filepath = 'best.h5'
save_model_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    verbose=1,
    save_best_only=True)

In [ ]:
#model = load_model('../input/tps1021/best_85221.h5')

In [ ]:
N_split = int(0.1 * len(dataset_train_sc))
dataset_sc_TRAIN = dataset_train_sc[:-N_split, :]
dataset_sc_VAL = dataset_train_sc[-N_split:, :]
y_TRAIN = y_sc[:-N_split]
y_VAL = y_sc[-N_split:]

train = tf.keras.preprocessing.timeseries_dataset_from_array(
    dataset_sc_TRAIN,
    y_TRAIN,
    sequence_length=lookback,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
)

val = tf.keras.preprocessing.timeseries_dataset_from_array(
    dataset_sc_VAL,
    y_VAL,
    sequence_length=lookback,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=batch_size,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
)

In [ ]:
train = (dataset_sc_TRAIN, y_TRAIN)
val = (dataset_sc_VAL, y_VAL)

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, min_lr=0.0005, verbose=1, mode='min')

In [ ]:

#EPOCHS = 500
#EPOCHS = 1
#model.fit(
#    dataset_sc_TRAIN, y_TRAIN, validation_data=(dataset_sc_VAL, y_VAL), batch_size=batch_size, epochs=EPOCHS, callbacks=[save_model_callback, reduce_lr], shuffle=True)


repeat few times 2 cells above

In [ ]:
#with strategy.scope():
#    model = load_model('best.h5')

In [ ]:
with strategy.scope():
    model = load_model('../input/ss-tsf/best_6.3584e-04-162.h5')

In [ ]:
preds = model.predict(dataset_test_sc)
preds1 = y_scaler.inverse_transform(preds)
preds1 = qty.inverse_transform(preds1)

In [ ]:
for i in range(len(preds1)): preds1[i] = 0 if preds1[i] < 0 else preds1[i]

In [ ]:
output = pd.DataFrame({'Id': test.index,'sales': preds1[:,0]})
path = 'sample_submission.csv'
output.to_csv(path, index=False)
output 